# Localisation of Mobile Agents & Simultaneous Localisation and Mapping

A step by step guide to LMA and SLAM (by Michael Fürst 2016-)

Maybe rather an in depth tutorial how to become an expert in slam.

The first part will be a collection of questions around localisation that one should be capable to answer.

The second part then will in detail (with python code examples) explain different slam approaches.

## Literature

Parts of this notepad are directly taken from my notes to the lecture "localisation of mobile agents" at KIT held by Dr. Kurz in 2016 (SS16).

Some knowledge is taken from the "slam course" by Cyrill Stachniss at Albert-Ludwigs-Universität Freiburg (WS13/14) and P.D.-I.C. Brenner. Lecture - slam b 02, November 2012 (both can be found on YouTube).

Highly Recommended Literature:
* G. Grisetti, R. Kummerle, C. Stacniss, and W. Burgard. A tutorial on graph-based slam. Intelligent Transportation Systems Magazine, IEEE 2(4):31-43, winter 2010.
* C. Stacniss, W. Burgard, M. Bennewitz, and K. Arras. Introduction to mobile robotics - iterative closest point algorithm, 2011.
* S. Thrun, W. Burgard, and D. Fox. Probabilistic Robotics (Intelligent Robotics and Autonomous Agents). The MIT Press, 2005.

Literature from which I might implicitly borrowed (in no specific order):
* A. Censi. An icp variant using a point-to-line metric. In Robotics and Automation, 2008. ICRA 2008. IEEE International Conference on, pages 19-25, May 2008.
* H.-P. Chiu, S. Williams, F. Dellaert, S. Samarasekera, and R. Kumar. Robust vision-aided navigation using sliding-window factor graphs. In Robotics and Automation (ICRA), 2013 IEEE International Conference on pages 46-53, May 2013.
* F. Dellaert. Factor graphs and gtsam: A hands-on introuction, september 2012.
* A. I. Eliazar and R. Parr. Learning Probabilistic Motion Models for Mobile Robots. Proceedings of the 21st International Conference on Machine Learning (ICML), 2004.
* F. A. Faruqi. Non-linear mathematical model for integrated global positioning/intertial navigation systems. Appl. Math. Comput., 115(2-3):191-212, Oct. 2000.
* U. Frese, P. Larson, and T. Duckett. A multilevel relaxation algorithm for simultaneous localization and mapping. IEEE Transactions on Robotics, 21(2):196-207, 2005.
* M. Kaess, A. Ranganathan, A. Dellaert, iSAM: Incremental smoothing and mapping. IEEE Transactions on Robotics, 24(6):1365-1378, 2008.
* H.-A. oeliger. An introduction to factor graphs. IEEE Signal Processing Magazine, 21(1):28-41, Jan 2004.
* D. Schleicher, L. Bergasa, M. Ocana, R. Barea, and E. Lope.. Real-time hierarchical gps aided visual slam on urban environments. In Robotics and automation, 2009. ICRA '09. IEEE International Conference on, pages 4381-4386, May 2009.


#### Some includes that are required everywhere...

In [1]:
import numpy as np

---


## Localisation of Mobile Agents

### General

#### What is localisation?

Localisation means estimating the pose (position and orientation) of a mobile agent.

#### How to evaluate the quality of a localisation procedure?

An estimator can be analyzed if it is unbiased or has low variance.
Sometimes konvergencespeed is a criteria too.

Basically answering the following questions:
How exact is the estimate?
Has it got any systematic errors? (bias)
Does it jump around? (variance)
Is it consistent?

A bonus that most people do not evaluate is:
How resistant is the procedure against errors on the input data?
Can it correct systematic errors in the input data?


#### What is the default approach to localisation?

The mobile agent has measurements for a given timestamp and tries to estimate its pose from those. There are several approaches that are partially based on each other: dead reckoning, static localisation, dynamic localisation and SLAM.



---

### Dead Reckoning (Odometry)

#### How does dead reckoning work and what are problems?

Dead Reckoning estimates the pose based on a starting pose using a motion model (e.g. Odometry).

The system equation with **pose $x$, controll $u$** and system matrices **$A$ and $B$** is:

$x \leftarrow A \cdot x + B \cdot u$.

The **covariance $C$** is updated as follows:

$C \leftarrow A C A^T + B Q B^T$

where **motion noise covariance $Q$** is used.

#### Which sensors are used?

1. wheelencoders -> distance
2. optical flow (camera) -> speed
3. IMU -> accelleration
4. servos -> steering angle
5. laserscans -> relative pose

#### Kinematics

##### What is an ideal wheel?

An ideal wheel only touches the ground in one point and has no slip, neither in driving direction nor orthogonally.

##### What is the ICR?

The ICR (instantaneous center of rotation) is the point around which a vehicle drives in a circle (at that certain time).

**Veocity $v$** in the robot system is translated into a **relative velocity $v_r$** given the **radius $r$** as follows:

$v_{robot} \leftarrow \left[ \begin{array}{cccc}
1\\
0\\
1/r\\ \end{array} \right] \cdot v$

And into the world coordinate system given the **global rotation $\theta$** of the robot:

$v_{global} \leftarrow \left[ \begin{array}{cccc}
\cos(\theta) & - \sin(\theta) & 0\\
\sin(\theta) & \cos(\theta) & 0\\
0 & 0 & 1\\ \end{array} \right] \cdot \left[ \begin{array}{cccc}
1\\
0\\
1/r\\ \end{array} \right] \cdot v = \left[ \begin{array}{cccc}
\cos(\theta)\\
\sin(\theta)\\
1/r\\ \end{array} \right] \cdot v$


##### How does the two wheel kinematics work?

The two wheels can run at different speeds **left velocity $v_l$** and **right velocity $v_r$**.
Sometimes this kinematic is also refered as tank drive.
The wheels are **distance $d$** appart from each other.

$v_{robot} = \left[ \begin{array}{cccc}
\frac{1}{2} & \frac{1}{2}\\
0 & 0\\
-\frac{1}{d} & \frac{1}{d}\\ \end{array} \right] \cdot \left[ \begin{array}{cccc}
v_l\\
v_r\\
\end{array} \right]$

$v_{global}(\theta) = \left[ \begin{array}{cccc}
\frac{\cos(\theta)}{2} & \frac{-\sin(\theta)}{2}\\
\frac{\sin(\theta)}{2} & \frac{\cos(\theta)}{2}\\
-\frac{1}{d} & \frac{1}{d}\\ \end{array} \right] \cdot \left[ \begin{array}{cccc}
v_l\\
v_r\\
\end{array} \right]$

##### How does dead reckoning work for two wheel vehicles?

Assume a time discrete model: $v_l(t), v_r(t)$ for $t \in N$.

Calculate $v_{robot}(t)$ using the formula above.

$x(t) = x(t-1) + v_{global}(\theta_{t-1}, t) \cdot \Delta t\\
= x(t-1) + \left[ \begin{array}{cccc}
\frac{\cos(\theta_{t-1})}{2} & \frac{-\sin(\theta_{t-1})}{2}\\
\frac{\sin(\theta_{t-1})}{2} & \frac{\cos(\theta_{t-1})}{2}\\
-\frac{1}{d} & \frac{1}{d}\\ \end{array} \right] \cdot \left[ \begin{array}{cccc}
v_l(t)\\
v_r(t)\\
\end{array} \right]$

So we can write this in the form of:

$x_t \leftarrow A \cdot x_{t-1} + B_t \cdot u_t$

with $A$ as the identity and $B_t = \left[ \begin{array}{cccc}
\frac{\cos(\theta_{t-1})}{2} & \frac{-\sin(\theta_{t-1})}{2}\\
\frac{\sin(\theta_{t-1})}{2} & \frac{\cos(\theta_{t-1})}{2}\\
-\frac{1}{d} & \frac{1}{d}\\ \end{array} \right]$ and $u_t = \left[ \begin{array}{cccc}
v_l(t)\\
v_r(t)\\
\end{array} \right]$

And with that the covariance is also defined we only need a $Q$ to actually calculate it ($C_0 = 0$ we know where we start exactly):

$C_t \leftarrow A C_t A^T + B_t Q B_t^T$

##### What does the uncertainty do for dead reckoning?

$C_t \leftarrow A C_t A^T + B_t Q B_t^T$

$\rightarrow$ It grows unbounded.

---

### Static Localisation

#### How does static localisation work?

The mobile agent measures its environment and can estimate its pose or position from the measurement.

The measurement equation is:

$y = H \cdot x$

minimize squared error:

$\rightarrow x = (H^T W^{-1} H) H^T W^{-1} \cdot y$

if $W^{-1} = I$ with the pseudo-inverse $H^+ = (H^T H) H^T$ it can be simplified to:

$\rightarrow x = H^+ y$

#### Which sensors are used?

GPS, Laserscanner, Ultrasonnic, Cameras (sometimes with markers), Beacons, Radar, WiFi, GSM, ...

#### What does uncertainty do?

$C = C - C H^T (W^{-1} + HCH^T)^{-1} HC$

$\rightarrow$ uncertainty decreases and converges to 0.

#### Examples for linear and non-linear static localisation?

Linear: Zug auf Schine -> Kilometerstelle ($B \cdot u$ unabh. von $x$)

Nicht Linear: Fahrzeug -> Pose (nicht lineares Bewegungsmodell)

#### Least-Squares (LQ)

##### How does least squares work (batch/recursive)?

Batch:

$x \leftarrow (H^T W^{-1} H)^{-1} H^T W^{-1} \cdot y$

$C \leftarrow (H^T W^{-1} H)^{-1}$

Recursive:

$x \leftarrow (I - KH) \cdot x + K \cdot y$

$C \leftarrow (I - KH) C (I - KH)^T + K W^{-1} K^T\\
     = C - C H^T(W^{-1} + H C H^T)^{-1} H C$
     
with $K = C H (W^{-1} + HCH^T)^{-1}$

##### Why least squares?

LQ can be written as a matrixproduct and allows the algebraic solution for batch and recursive to be found:

$G(x) = e \cdot e = (y - Hx)^T W^{-1} (y-Hx)$
    
Minimize $G(x)$

##### Why is a diagonal weight matrix sufficient?

Position of the landmarks is known and landmarks are independant. Just like the error in x and y direction.

##### Why recursive least squares?

Batch-LQ has a complexity of $O(n^3)$ whenever a new measurement arrives.

Recursive-LQ in contrast has the complexity $O(1)$ to add a new measurement.

##### What is the structure of recursive LQ?

$x \leftarrow (I - KH) \cdot x + K \cdot y = x + K(y-Hx)$

$C \leftarrow (I - KH) C (I - KH)^T + K W^{-1} K^T$
     
with $K = C H (W^{-1} + HCH^T)^{-1}$

##### How to initialize LQ?

Randomly initialize x and give it the correct covariance.

$x_0 \leftarrow rand\\
C_0 \leftarrow \infty$

However, that does not work so well in practical applications.
alternative: (with first measurement)

$x_0 = Hy_0\\
C_0 \leftarrow = C_{y_0}$

alternative: (with batch-lq)

#### How to addres angles?

Bsp.: $\mu \{ 350°, 10° \} = \frac{1}{2} \cdot ( 350° + 10° ) = \frac{1}{2} \cdot 360° = 180° \rightarrow$ WRONG!

$\arctan(\mu{ \left[ \begin{array}{cccc}
\cos(350°)\\
\sin(350°)\\
\end{array} \right], \left[ \begin{array}{cccc}
\cos(10°)\\
\sin(10°)\\
\end{array} \right] })$

$= \arctan( \frac{1}{2} \cdot ( \left[ \begin{array}{cccc}
\cos(350°)\\
\sin(350°)\\
\end{array} \right] + \left[ \begin{array}{cccc}
\cos(10°)\\
\sin(10°)\\
\end{array} \right] )) = \arctan(\left[ \begin{array}{cccc}
\cos(10°)\\
0\\
\end{array} \right]) = 0° \rightarrow$ correct

#### Deterministic Localisation

##### What common measurement functions exist?

* Triangulation: angle and distance known
* Trilateration: 3 distances known

##### What advantages do iterative and analytical solitions have?

iterative numerical:
* $+$ simpler
* $+$ recursive updates
* $-$ only correct in the limit

analytical:
* $+$ exact result
* $-$ not allways possible

##### How to address non linear systems?

Linearisation: (Taylor)

$g(x_0 + \Delta x) = g(x_0) + G \cdot \Delta x$

with $G = \nabla g(x_0) \leftarrow$ Jacobi Matrix in $x_0$

##### What are Quaternions? Pros and Cons?

$q(a,b,c,d) = a + bi + cj + dk, i^2 = -1, j^2 = -1, k^2=-1$

$q(\alpha, u) = \cos(\frac{\alpha}{2}) + \sin(\frac{\alpha}{2}) \cdot (u_x i + u_y i + u_z k)$

Rotation: $p(\alpha, u) \cdot p \cdot \bar{q}(\alpha, u)$ where $p = 0 + p_x i + p_y j + p_z k$

Advantages:
* No singularity (compare gimbal lock)
* Low redundancy (compare to rotation matrix)
* simple to invert (it's the complex conjugate)

Issues:
* None?
* Maybe not so intuitive?

---

### Dynamic Localisation

#### What can you do?

1. Only dead reckoning **or** static localisation $\rightarrow$ typically **not good**

2. Dead reckoning **and** static localisation with an early fusion $\rightarrow$ **good**

#### How do static localisation and dead reckoning profit from each other?

Prediction: dead reckoning $\rightarrow$ uncertainty grows, position is moved

Correction: static localisation $\rightarrow$ uncertanty shrinks, improvement of the current position.

Uncertainty ideally stays within a margin.

#### How to benefit the most out of both?

Alternate between dead reckoning and static localisation. It keeps the Covariance minimal.

#### Recursive Localisation what is the principal?

correction, combination of estimators, prediction

#### Kalman Filter (KF)

Prediction:

$x \leftarrow Ax + Bu$

$C \leftarrow ACA^T + BQB^T$ with $Q$ motion noise

Correction:

$K \leftarrow C H^T(C_m + HCH^T)^{-1}$ with $C_m$ measurement noise

$x \leftarrow x + K(y-Hx)$

$C \leftarrow C - C H^T(C_m + K C H^T)^{-1} H C$

#### How to find the kalman gain?

TODO is this actually important?

#### How to compare covariance matrices?

Trace $\approx$ circumference of the elipse (of the covariance matrix)

Determinant $\approx$ area of the elipse ( of the covariance matrix)

#### When is the kalman filter optimal?

In the linear case.

Or when a problem has a normal distribution.

#### What are the requirements for a kalman filter?

* linear
* normal distribution


#### What is the difference between KF and recursive LQ?

LQ has weightmatrices, whereas KF uses covariance matrices.

---

# SLAM

## Basics

### What is SLAM?

Simultaneous localisation and mapping is a category of algorithms that localise a mobile agent and create a map of the environment (which is previously unknown) at the same time.

The pose and the map are unknown and improve each othere.

### What basic approaches exist?

Filtering:

* EKF (extended kalman filter)
* UKF (uncented kalman filter)
* Particlefilter (eg FastSLAM)

Smoothing:
* Grpahbased: Factorgraphs and Posegraphs
* Multilevel Relaxation (MLR)

### What are the challanges?

* "chicken egg"-problem: map and localisation are dependant on each other
* usually non linear-problem (sometimes even multimodal)
* matching with landmarks (in the map) is difficult

### What is loop closure and why is it important?

Whereas the map is unknown the error grows over time (like prediction).

If the robot sees a landmark that it has seen a while ago, it is called loop-closure. Similar to static localisation the robot can improve its position estimate with the map (like the correction step).

## Probability theory [WIP]

### Distributions

#### Continuous Distributions

A continuous distribution has a distribution function. Sometimes called pdf.

In [ ]:
def f_gaussian(x, mu, sigma):
    return 1.0 / math.sqrt(2 * math.pi * sigma * sigma) * math.exp(-(x-mu)*(x-mu)/(2*sigma*sigma))

def F_gaussian(x, mu, sigma):
    return 1.0 / 2.0 * (1 + math.erf((x-mu) * math.sqrt(2 * sigma * sigma)))

### Discrete Distributions

TODO

---

## Algorithms needed for SLAM

### Calculate the mean of a set

Sometimes it is very handy to have a function that calculates the position mean of a set.

In [2]:
def calc_pos_mean(arr):
    n = len(arr)
    mean = [0, 0]
    for x in arr:
        mean[0] += x[0]
        mean[1] += x[1]
    mean[0] = mean[0] / float(n)
    mean[1] = mean[1] / float(n)
    return mean

def calc_pose_mean(arr):
    mean = calc_pos_mean(arr)
    rot_arr = []
    for p in arr:
        rot_arr.append([cos(p[2]), sin(p[2])])
    rot_mean = calc_pos_mean(rot_arr)
    return [mean[0], mean[1], atan2(rot_mean[1], rot_mean[0])]

### Feature Transform

The feature transform algorithm is a basic algorithm, that calculates how to match two ordered lists of points with the least squared error.
As the name suggests, it can be used to match two lists of features.
Therfore it is very important for several slam solutions, for example ICP can use this algorithm.

It calculates how the measurements have to be transformed to match the reference with the least error.
An assumption is that for each measurement m_i there is a reference r_i with the same index.

Calculate the transform:

In [3]:
def feature_transform(measurements, reference, fixed_scale=True):
    l = measurements
    r = reference
    n = len(l)
    
    # Calculate the means
    l_mean = calc_pos_mean(l)
    r_mean = calc_pos_mean(r)
    
    # Calculate the coefficients
    cs = 0.0
    ss = 0.0
    rr = 0.0
    rl = 0.0
    for i in range(n):
        l_i[0] = l[i][0] - l_mean[0]
        l_i[1] = l[i][1] - l_mean[1]
        r_i[0] = r[i][0] - r_mean[0]
        r_i[1] = r[i][1] - r_mean[1]
        
        cs +=   r_i[0] * l_i[0] + r_i[1] * l_i[1]
        ss += - r_i[0] * l_i[1] + r_i[1] * l_i[0]
        rr +=   r_i[0] * r_i[0] + r_i[1] * r_i[1]
        ll +=   l_i[0] * l_i[0] + l_i[1] * l_i[1]
    
    scale = 1
    if not fixed_scale:
        scale = sqrt(rr / ll)
    
    denom = sqrt(cs * cs + ss * ss)
    c = cs / denom
    s = ss / denom
    
    alpha = atan2(s, c)
    tx = r_mean[0] - scale * (c * l_mean[0] - s * l_mean[1])
    ty = r_mean[1] - scale * (s * l_mean[0] + c * l_mean[1])
    
    return (tx, ty, alpha, scale)

The calculated feature transform can be applied to a pose with a simple calculation.

**$p$** is the position of the pose in 2-d.
**$\theta$** is the rotation of the pose as an angle.
**$t$** is the translation vector of the feature transform $(t_x, t_y)^T$.
**$\alpha$** the rotation angle alpha resulting in **$R(\alpha)$** as the rotation matrix for alpha.
**$s$** the scale.

$p \leftarrow s \cdot (R(\alpha) \cdot p) + t$

$r \leftarrow \theta + \alpha$

And the code:

In [4]:
def apply_transform(pose, transform):
    x_off, y_off, alpha, scale = transform
    x, y, theta = pose
    
    c = cos(alpha)
    s = sin(alpha)
    tx = (x * c - y * s) * scale + x_off
    ty = (x * s + y * c) * scale + y_off
    
    return (tx, ty, theta + alpha)

### Convolution

Because we are only interested in convoluting a 1d kernel we can simplify it a bit.

The convolute_1d implements a convolution for 1d case.

The convolute_1d_arr extends the 1d convolution to be applied on multiple dimensions at once instead of splitting them applying a convolute_1d and the joining them.

N-d Convolution for N other than 1 is not implemented. We won't need it for SLAM.

In [5]:
# Asserts len(kernel) % 2 = 1
def convolute_1d(arr, kernel):
    result = []
    n_a = len(arr)
    n_k = len(kernel)
    for i in range(n_a):
        result.append(0)
        for k in range(n_k):
            p_i = i + k - int(n_k / 2)
            if p_i < 0:
                p_i = 0
            if p_i >= n_a:
                p_i = n_a - 1
                
            result[i] += arr[p_i] * kernel[k]
    return result

# Asserts len(kernel) % 2 = 1
# Asserts foreach i, j: len(arr[i]) = len(arr[j])
def convolute_1d_arr(arr, kernel):
    result = []
    n_a = len(arr)
    n_k = len(kernel)
    dim = len(arr[0])
    for i in range(n_a):
        result.append([])
        for l in range(dim):
            result[i].append(0)
        for k in range(n_k):
            p_i = i + k - int(n_k / 2)
            if p_i < 0:
                p_i = 0
            if p_i >= n_a:
                p_i = n_a - 1
                
            for l in range(dim):
                result[i][l] += arr[p_i][l] * kernel[k]
    return result

### Smooth Poses & Points

Sometimes it is handy to have an algorithm that smoothes poses and points.

I propose to use a simple smoothing using a kernel. This basically convolutes the points with a weight kernel several times.

```
for x iterations do:
    points = convolute(points, weights)
```

e.g. weights = [0.25, 0.5, 0.25], iterations = 1

In [6]:
# Asserts sum of weights = 1
# Asserts len(weights) % 2 = 1
# Asserts foreach i, j: len(points[i]) = len(points[j])
def smooth_points(points, weights, iterations):
    for it in range(iterations):
        points = convolute_1d_arr(points, weights)
    return points

# Asserts the sum of weights is 1
# Asserts len(weights) % 2 = 1
# Asserts 2d poses
def smooth_poses(poses, weights, iterations):
    smoothable_poses = []
    for x in poses:
        smoothable_poses.append([x[0], x[1], cos(x[2]), sin(x[2])])
    smoothed = smooth_points(smoothable_poses, weights, iterations)
    poses = []
    for x in smoothed:
        poses.append([x[0], x[1], atan2(x[3], x[2])])
    return poses

### Example

In [11]:
print("TODO")

TODO
